In [1]:
from google.colab import drive
import os
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd drive/My\ Drive/IR/Project/

/content/drive/My Drive/IR/Project


In [3]:
%ls

data/  flant5_citation_identification/  flant5_news_cat/


In [4]:
!pip install transformers[sentencepiece]
!pip install sentencepiece
!pip install datasets
!pip install rank_bm25
!pip install accelerate
!pip install -U bertopic
!pip install -U safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 17.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_

In [5]:
import torch
import json
import re

from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModel
from rank_bm25 import BM25Okapi

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, \
    Seq2SeqTrainer
from datasets import Dataset, load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
print(f'Device: {device}')

Device: cuda


# Models

In [8]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
# topic_model = BERTopic.load("MaartenGr/BERTopic_ArXiv")

# contriever_tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
# contriever_model = AutoModel.from_pretrained('facebook/contriever')

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

#JSON Read/Write

In [9]:
def read_json_dict(filename: str):
    with open(filename) as fp:
        return json.load(fp)

#Load Retrieved & Preprocessed Citation Identification Data

In [10]:
#Model 1
user_citation_bm25_k_1_train = read_json_dict('data/citation_identification/raw/bm25/k_1/train.json')
user_citation_bm25_k_1_dev =  read_json_dict('data/citation_identification/raw/bm25/k_1/dev.json')

#Model 2
user_citation_bm25_k_4_train = read_json_dict('data/citation_identification/raw/bm25/k_4/train.json')
user_citation_bm25_k_4_dev = read_json_dict('data/citation_identification/raw/bm25/k_4/dev.json')

#Model 3
user_citation_topic_model_k_1_train = read_json_dict('data/citation_identification/raw/topic_model/k_1/train.json')
user_citation_topic_model_k_1_dev = read_json_dict('data/citation_identification/raw/topic_model/k_1/dev.json')

#Model 4
user_citation_topic_model_k_4_train = read_json_dict('data/citation_identification/raw/topic_model/k_4/train.json')
user_citation_topic_model_k_4_dev = read_json_dict('data/citation_identification/raw/topic_model/k_4/dev.json')

#Model 5
time_citation_bm25_k_1_train = read_json_dict('data/time_citation_identification/raw/bm25/k_1/train.json')
time_citation_bm25_k_1_dev = read_json_dict('data/time_citation_identification/raw/bm25/k_1/dev.json')

#Model 6
time_citation_bm25_k_4_train = read_json_dict('data/time_citation_identification/raw/bm25/k_4/train.json')
time_citation_bm25_k_4_dev = read_json_dict('data/time_citation_identification/raw/bm25/k_4/dev.json')

#Model 7
time_citation_topic_model_k_1_train = read_json_dict('data/time_citation_identification/raw/topic_model/k_1/train.json')
time_citation_topic_model_k_1_dev = read_json_dict('data/time_citation_identification/raw/topic_model/k_1/dev.json')

#Model 8
time_citation_topic_model_k_4_train = read_json_dict('data/time_citation_identification/raw/topic_model/k_4/train.json')
time_citation_topic_model_k_4_dev = read_json_dict('data/time_citation_identification/raw/topic_model/k_4/dev.json')

#Model 9
user_citation_rerank_cont_k_1_train = read_json_dict('data/citation_identification/raw/rerank_cont/k_1/train.json')
user_citation_rerank_cont_k_1_dev =  read_json_dict('data/citation_identification/raw/rerank_cont/k_1/dev.json')

#Model 10
user_citation_rerank_cont_k_4_train = read_json_dict('data/citation_identification/raw/rerank_cont/k_4/train.json')
user_citation_rerank_cont_k_4_dev =  read_json_dict('data/citation_identification/raw/rerank_cont/k_4/dev.json')

#Model 11
time_citation_rerank_cont_k_1_train = read_json_dict('data/time_citation_identification/raw/rerank_cont/k_1/train.json')
time_citation_rerank_cont_k_1_dev =  read_json_dict('data/time_citation_identification/raw/rerank_cont/k_1/dev.json')

#Model 12
time_citation_rerank_cont_k_4_train = read_json_dict('data/time_citation_identification/raw/rerank_cont/k_4/train.json')
time_citation_rerank_cont_k_4_dev =  read_json_dict('data/time_citation_identification/raw/rerank_cont/k_4/dev.json')


#Model 13
user_citation_cluster_k_1_train = read_json_dict('data/citation_identification/raw/cluster/k_1/train.json')
user_citation_cluster_k_1_dev =  read_json_dict('data/citation_identification/raw/cluster/k_1/dev.json')

#Model 14
user_citation_cluster_k_4_train = read_json_dict('data/citation_identification/raw/cluster/k_4/train.json')
user_citation_cluster_k_4_dev =  read_json_dict('data/citation_identification/raw/cluster/k_4/dev.json')

#Model 15
time_citation_cluster_k_1_train = read_json_dict('data/time_citation_identification/raw/cluster/k_1/train.json')
time_citation_cluster_k_1_dev =  read_json_dict('data/time_citation_identification/raw/cluster/k_1/dev.json')

#Model 16
time_citation_cluster_k_4_train = read_json_dict('data/time_citation_identification/raw/cluster/k_4/train.json')
time_citation_cluster_k_4_dev =  read_json_dict('data/time_citation_identification/raw/cluster/k_4/dev.json')

# Load Data

In [11]:
max_token_length = 512

def load_and_merge_files_citation_identification(raw_data, tokenizer):
    model_inputs = tokenizer(raw_data['input'], max_length=max_token_length, padding=True, truncation=True, return_tensors="pt").to(device)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(raw_data['target'], max_length=max_token_length, padding=True, truncation=True, return_tensors="pt").to(device)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs, raw_data

In [12]:
def get_dataset(raw_train, raw_dev):
    train_data, raw_data_train = load_and_merge_files_citation_identification(raw_train, tokenizer)
    dev_data, raw_data_dev = load_and_merge_files_citation_identification(raw_dev, tokenizer)

    train_dataset = Dataset.from_dict(train_data)
    dev_dataset = Dataset.from_dict(dev_data)

    return raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset

#Trainer

In [13]:
def calculate_accuracy(trainedmodel, data):
  length = len(data['input'])
  correct = 0
  total = 0

  for i in range(length):
    input_ids = tokenizer(data['input'][i], max_length=max_token_length, padding=True, truncation=True, return_tensors="pt").input_ids.to(device=device)
    out = trainedmodel.generate(input_ids.to(device=device)).to(device=device)

    output = tokenizer.decode(out[0]).replace('<pad>', '').replace('</s>', '').strip()
    if output == data['target'][i].strip():
      correct = correct + 1
    total = total + 1
  acc = correct / total
  print('Accuracy: {}'.format(acc))

def train_citation_identification(train_dataset, dev_dataset, trainmodel, traintokenizer, train_data_collator):
    training_args = Seq2SeqTrainingArguments(
        output_dir="flant5_citation_identification",
        save_strategy="no",
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=2,
        weight_decay=1e-4,
        save_total_limit=3,
        num_train_epochs=5,
        warmup_ratio=0.05,
        fp16=False,
    )

    trainer = Seq2SeqTrainer(
        model=trainmodel,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
        tokenizer=traintokenizer,
        data_collator=train_data_collator
    )

    trainer.train()

    return trainer, trainmodel

# Model 1 (User + BM25 + K1)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset = get_dataset(user_citation_bm25_k_1_train, user_citation_bm25_k_1_dev)

trainer, trainedmodel = train_citation_identification(train_dataset, dev_dataset, model, tokenizer, data_collator)

Epoch,Training Loss,Validation Loss
1,0.187600,0.167835
2,0.168500,0.159984
3,0.162400,0.164055
4,0.145900,0.171851
5,0.136500,0.187442


In [ ]:
trainer.evaluate()

{'eval_loss': 0.18744158744812012,
 'eval_runtime': 30.6068,
 'eval_samples_per_second': 49.009,
 'eval_steps_per_second': 24.504,
 'epoch': 5.0}

In [ ]:
calculate_accuracy(trainedmodel, raw_data_dev)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Accuracy: 0.65


# Model 2 (User + BM25 + K4)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset = get_dataset(user_citation_bm25_k_4_train, user_citation_bm25_k_4_dev)

trainer, trainedmodel = train_citation_identification(train_dataset, dev_dataset, model, tokenizer, data_collator)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.179800,0.159131
2,0.152400,0.152583
3,0.142000,0.163765
4,0.119600,0.182441
5,0.112400,0.194716


In [ ]:
trainer.evaluate()

{'eval_loss': 0.1947159767150879,
 'eval_runtime': 39.139,
 'eval_samples_per_second': 38.325,
 'eval_steps_per_second': 19.162,
 'epoch': 5.0}

In [ ]:
calculate_accuracy(trainedmodel, raw_data_dev)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Accuracy: 0.6693333333333333


#Model 3 (User + TopicModel + K1)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset = get_dataset(user_citation_topic_model_k_1_train, user_citation_topic_model_k_1_dev)

trainer, trainedmodel = train_citation_identification(train_dataset, dev_dataset, model, tokenizer, data_collator)

Epoch,Training Loss,Validation Loss
1,0.185600,0.166287
2,0.168400,0.161034
3,0.165600,0.165855
4,0.152000,0.177300
5,0.144000,0.183043


In [ ]:
trainer.evaluate()

{'eval_loss': 0.18304282426834106,
 'eval_runtime': 27.2041,
 'eval_samples_per_second': 55.139,
 'eval_steps_per_second': 27.569,
 'epoch': 5.0}

In [ ]:
calculate_accuracy(trainedmodel, raw_data_dev)

Accuracy: 0.6266666666666667


# Model 4 (User + TopicModel + K4)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset = get_dataset(user_citation_topic_model_k_4_train, user_citation_topic_model_k_4_dev)

trainer, trainedmodel = train_citation_identification(train_dataset, dev_dataset, model, tokenizer, data_collator)

Epoch,Training Loss,Validation Loss
1,0.188000,0.169808
2,0.169300,0.161321
3,0.166800,0.165018
4,0.155300,0.163993
5,0.145600,0.179003


In [ ]:
trainer.evaluate()

{'eval_loss': 0.17900317907333374,
 'eval_runtime': 39.7189,
 'eval_samples_per_second': 37.765,
 'eval_steps_per_second': 18.883,
 'epoch': 5.0}

In [ ]:
calculate_accuracy(trainedmodel, raw_data_dev)

Accuracy: 0.6393333333333333


#Model 5 (Time + BM25 + K1)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset = get_dataset(time_citation_bm25_k_1_train, time_citation_bm25_k_1_dev)

trainer, trainedmodel = train_citation_identification(train_dataset, dev_dataset, model, tokenizer, data_collator)

Epoch,Training Loss,Validation Loss
1,0.187600,0.167835
2,0.168500,0.159984
3,0.162400,0.164055
4,0.145900,0.171851
5,0.136500,0.187442


In [ ]:
trainer.evaluate()

{'eval_loss': 0.18744158744812012,
 'eval_runtime': 32.1389,
 'eval_samples_per_second': 46.672,
 'eval_steps_per_second': 23.336,
 'epoch': 5.0}

In [ ]:
calculate_accuracy(trainedmodel, raw_data_dev)

Accuracy: 0.65


#Model 6 (Time + BM25 + K4)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset = get_dataset(time_citation_bm25_k_4_train, time_citation_bm25_k_4_dev)

trainer, trainedmodel = train_citation_identification(train_dataset, dev_dataset, model, tokenizer, data_collator)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.179800,0.159131
2,0.152400,0.152583
3,0.142000,0.163765
4,0.119600,0.182441
5,0.112400,0.194716


In [ ]:
trainer.evaluate()

{'eval_loss': 0.1947159767150879,
 'eval_runtime': 38.6689,
 'eval_samples_per_second': 38.791,
 'eval_steps_per_second': 19.395,
 'epoch': 5.0}

In [ ]:
calculate_accuracy(trainedmodel, raw_data_dev)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Accuracy: 0.6693333333333333


# Model 7 (Time + TopicModel + K1)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset = get_dataset(time_citation_topic_model_k_1_train, time_citation_topic_model_k_1_dev)

trainer, trainedmodel = train_citation_identification(train_dataset, dev_dataset, model, tokenizer, data_collator)

Epoch,Training Loss,Validation Loss
1,0.185600,0.166287
2,0.168400,0.161034
3,0.165600,0.165855
4,0.152000,0.177300
5,0.144000,0.183043


In [ ]:
trainer.evaluate()

{'eval_loss': 0.18304282426834106,
 'eval_runtime': 26.7797,
 'eval_samples_per_second': 56.013,
 'eval_steps_per_second': 28.006,
 'epoch': 5.0}

In [ ]:
calculate_accuracy(trainedmodel, raw_data_dev)

Accuracy: 0.6266666666666667


#Model 8 (Time + TopicModel + K4)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset = get_dataset(time_citation_topic_model_k_4_train, time_citation_topic_model_k_4_dev)

trainer, trainedmodel = train_citation_identification(train_dataset, dev_dataset, model, tokenizer, data_collator)

Epoch,Training Loss,Validation Loss
1,0.188000,0.169808
2,0.169300,0.161321
3,0.166800,0.165018
4,0.155300,0.163993
5,0.145600,0.179003


In [ ]:
trainer.evaluate()

{'eval_loss': 0.17900317907333374,
 'eval_runtime': 39.3641,
 'eval_samples_per_second': 38.106,
 'eval_steps_per_second': 19.053,
 'epoch': 5.0}

In [ ]:
calculate_accuracy(trainedmodel, raw_data_dev)

Accuracy: 0.6393333333333333


# Model 9 (User+RerankCont+K1)



In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset = get_dataset(user_citation_rerank_cont_k_1_train, user_citation_rerank_cont_k_1_dev)

trainer, trainedmodel = train_citation_identification(train_dataset, dev_dataset, model, tokenizer, data_collator)

Epoch,Training Loss,Validation Loss
1,0.187300,0.167385
2,0.170100,0.160280
3,0.165300,0.167485
4,0.152100,0.170445
5,0.143800,0.176814


In [ ]:
trainer.evaluate()

{'eval_loss': 0.1768142282962799,
 'eval_runtime': 27.3727,
 'eval_samples_per_second': 54.799,
 'eval_steps_per_second': 27.4,
 'epoch': 5.0}

In [ ]:
calculate_accuracy(trainedmodel, raw_data_dev)

Accuracy: 0.6506666666666666


# Model 10 (User+RerankCont+K4)

In [14]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset = get_dataset(user_citation_rerank_cont_k_4_train, user_citation_rerank_cont_k_4_dev)

trainer, trainedmodel = train_citation_identification(train_dataset, dev_dataset, model, tokenizer, data_collator)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.184900,0.158030
2,0.150100,0.153230
3,0.140400,0.166529
4,0.118300,0.169298
5,0.112500,0.189578


In [15]:
trainer.evaluate()

{'eval_loss': 0.18957844376564026,
 'eval_runtime': 38.6555,
 'eval_samples_per_second': 38.804,
 'eval_steps_per_second': 19.402,
 'epoch': 5.0}

In [16]:
calculate_accuracy(trainedmodel, raw_data_dev)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Accuracy: 0.6686666666666666


# Model 11 (time+RerankCont+K1)

In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset = get_dataset(time_citation_rerank_cont_k_1_train, time_citation_rerank_cont_k_1_dev)

trainer, trainedmodel = train_citation_identification(train_dataset, dev_dataset, model, tokenizer, data_collator)

Epoch,Training Loss,Validation Loss
1,0.187300,0.167385
2,0.170100,0.160280
3,0.165300,0.167485
4,0.152100,0.170445
5,0.143800,0.176814


In [18]:
trainer.evaluate()

{'eval_loss': 0.1768142282962799,
 'eval_runtime': 27.5856,
 'eval_samples_per_second': 54.376,
 'eval_steps_per_second': 27.188,
 'epoch': 5.0}

In [19]:
calculate_accuracy(trainedmodel, raw_data_dev)

Accuracy: 0.6506666666666666


# Model 12 (time+RerankCont+K4)

In [20]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset = get_dataset(time_citation_rerank_cont_k_4_train, time_citation_rerank_cont_k_4_dev)

trainer, trainedmodel = train_citation_identification(train_dataset, dev_dataset, model, tokenizer, data_collator)

Epoch,Training Loss,Validation Loss
1,0.184900,0.158030
2,0.150100,0.153230
3,0.140400,0.166529
4,0.118300,0.169298
5,0.112500,0.189578


In [21]:
trainer.evaluate()

{'eval_loss': 0.18957844376564026,
 'eval_runtime': 38.6955,
 'eval_samples_per_second': 38.764,
 'eval_steps_per_second': 19.382,
 'epoch': 5.0}

In [22]:
calculate_accuracy(trainedmodel, raw_data_dev)

Accuracy: 0.6686666666666666


# Model 13 (User+Cluster+K1)

In [14]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset = get_dataset(user_citation_cluster_k_1_train, user_citation_cluster_k_1_dev)

trainer, trainedmodel = train_citation_identification(train_dataset, dev_dataset, model, tokenizer, data_collator)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.186800,0.166692
2,0.160000,0.160248
3,0.154400,0.172623
4,0.136600,0.183882
5,0.124400,0.194811


In [15]:
trainer.evaluate()

{'eval_loss': 0.19481147825717926,
 'eval_runtime': 24.331,
 'eval_samples_per_second': 61.65,
 'eval_steps_per_second': 30.825,
 'epoch': 5.0}

In [16]:
calculate_accuracy(trainedmodel, raw_data_dev)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Accuracy: 0.6426666666666667


# Model 14 (User+Cluster+K4)

In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset = get_dataset(user_citation_cluster_k_4_train, user_citation_cluster_k_4_dev)

trainer, trainedmodel = train_citation_identification(train_dataset, dev_dataset, model, tokenizer, data_collator)

Epoch,Training Loss,Validation Loss
1,0.180300,0.149280
2,0.150200,0.145036
3,0.139700,0.157113
4,0.117500,0.174198
5,0.105400,0.178902


In [18]:
trainer.evaluate()

{'eval_loss': 0.1789024919271469,
 'eval_runtime': 24.7923,
 'eval_samples_per_second': 60.503,
 'eval_steps_per_second': 30.251,
 'epoch': 5.0}

In [19]:
calculate_accuracy(trainedmodel, raw_data_dev)

Accuracy: 0.6926666666666667


# Model 15 (Time+Cluster+K1)

In [20]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset = get_dataset(time_citation_cluster_k_1_train, time_citation_cluster_k_1_dev)

trainer, trainedmodel = train_citation_identification(train_dataset, dev_dataset, model, tokenizer, data_collator)

Epoch,Training Loss,Validation Loss
1,0.186400,0.163423
2,0.162800,0.160013
3,0.153700,0.180678
4,0.137200,0.183045
5,0.128600,0.191556


In [21]:
trainer.evaluate()

{'eval_loss': 0.19155560433864594,
 'eval_runtime': 23.5947,
 'eval_samples_per_second': 63.574,
 'eval_steps_per_second': 31.787,
 'epoch': 5.0}

In [22]:
calculate_accuracy(trainedmodel, raw_data_dev)

Accuracy: 0.6333333333333333


# Model 16 (Time+Cluster+K4)

In [23]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

raw_data_train, raw_data_dev, train_data, dev_data, train_dataset, dev_dataset = get_dataset(time_citation_cluster_k_4_train, time_citation_cluster_k_4_dev)

trainer, trainedmodel = train_citation_identification(train_dataset, dev_dataset, model, tokenizer, data_collator)

Epoch,Training Loss,Validation Loss
1,0.182200,0.155835
2,0.150100,0.145240
3,0.137200,0.169078
4,0.118500,0.163191
5,0.109700,0.179738


In [24]:
trainer.evaluate()

{'eval_loss': 0.17973800003528595,
 'eval_runtime': 24.2976,
 'eval_samples_per_second': 61.735,
 'eval_steps_per_second': 30.867,
 'epoch': 5.0}

In [25]:
calculate_accuracy(trainedmodel, raw_data_dev)

Accuracy: 0.6953333333333334
